# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
weather = pd.read_csv('Output_data/weather.csv', encoding="utf-8")
weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Alfred,-33.5906,26.8910,66.00,92,90,3.00,ZA,1615851698
1,1,Mataura,-46.1927,168.8643,55.99,69,74,21.99,NZ,1615851699
2,2,Busselton,-33.6500,115.3333,66.99,48,56,5.01,AU,1615851582
3,3,Fergus Falls,46.2830,-96.0776,32.00,80,75,10.36,US,1615851699
4,4,Bluff,-46.6000,168.3333,55.99,69,74,21.99,NZ,1615851700


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
loc = weather[['Lat','Lng']]
humidity = weather['Humidity'].astype(float)
#loc
#humidity

In [5]:
fig = gmaps.figure(center = [0,0],zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations = loc, 
                                 weights=humidity,
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#  * A max temperature lower than 80 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.
#  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
narrow_df = weather.loc[(weather["Max Temp"] > 70) 
                        & (weather["Max Temp"] < 80) 
                        & (weather["Wind Speed"] < 10) 
                        & (weather["Cloudiness"] == 0) ]
narrow_df = narrow_df.dropna(how='any')
narrow_df.reset_index(inplace=True)
del narrow_df['index']

narrow_df.head()
#len(narrow_df)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,30,Saint-Philippe,-21.3585,55.7679,77.00,78,0,4.61,RE,1615851705
1,124,Mar del Plata,-38.0023,-57.5575,72.00,73,0,6.91,AR,1615851510
2,158,Saint-Pierre,-21.3393,55.4781,77.00,78,0,4.61,RE,1615851739
3,184,Oussouye,12.4850,-16.5469,71.46,49,0,8.77,SN,1615851747
4,197,Poum,-20.2333,164.0167,79.93,70,0,2.64,NC,1615851751


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#URL:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type":"hotel", "keyword":"hotel", "radius":5000, "key":g_key}

In [8]:
print('-------------------')
print('---Hotel Results---')
print('-------------------')

#for loop
for index, row in narrow_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    params['location'] = f'{lat},{lng}'
    #params['location'] = f"{lat},{lng}"
    
    #Make API requests
    print(f"Getting Results for Index {index}:{city}.")
    response = requests.get(url, params=params).json()
    results = response["results"]
    
    #Add hotel to narrow_df
    try:       
        print(f"Closet hotel in {city} is {results[0]['name']}.")
        narrow_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Oh Snap, no hotels... SKIPPING!")
        
    print('------------------')
print('----End Report----')
print('------------------')

-------------------
---Hotel Results---
-------------------
Getting Results for Index 0:Saint-Philippe.
Closet hotel in Saint-Philippe is Les Embruns Du Baril.
------------------
Getting Results for Index 1:Mar del Plata.
Closet hotel in Mar del Plata is NH Gran Hotel Provincial.
------------------
Getting Results for Index 2:Saint-Pierre.
Closet hotel in Saint-Pierre is LODGE PALMAE.
------------------
Getting Results for Index 3:Oussouye.
Closet hotel in Oussouye is Campement ALULUM.
------------------
Getting Results for Index 4:Poum.
Oh Snap, no hotels... SKIPPING!
------------------
Getting Results for Index 5:Ségou.
Closet hotel in Ségou is Hôtel Mivera.
------------------
Getting Results for Index 6:Adrar.
Oh Snap, no hotels... SKIPPING!
------------------
Getting Results for Index 7:Esmeraldas.
Closet hotel in Esmeraldas is Parque do Avestruz Hotel Fazenda.
------------------
Getting Results for Index 8:Tessalit.
Oh Snap, no hotels... SKIPPING!
------------------
Getting Result

In [9]:
narrow_df = narrow_df.dropna(how='any')
narrow_df.head()
#len(narrow_df)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,30,Saint-Philippe,-21.3585,55.7679,77.00,78,0,4.61,RE,1615851705,Les Embruns Du Baril
1,124,Mar del Plata,-38.0023,-57.5575,72.00,73,0,6.91,AR,1615851510,NH Gran Hotel Provincial
2,158,Saint-Pierre,-21.3393,55.4781,77.00,78,0,4.61,RE,1615851739,LODGE PALMAE
3,184,Oussouye,12.4850,-16.5469,71.46,49,0,8.77,SN,1615851747,Campement ALULUM
5,238,Ségou,13.4317,-6.2157,77.02,17,0,6.96,ML,1615851762,Hôtel Mivera


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_df.iterrows()]
hotel = narrow_df[["Lat", "Lng"]]



In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))